In [7]:
# Modules
import gurobipy as grb
import pandas as pd
import numpy as np
import math
import random

# Standard output divider
div = "-----"

# Print warning message


# Timeslots
alltimeslots = {
    1: "8:00am",
    2: "8:45am",
    3: "9:30am",
    4: "10:30am",
    5: "11:15am",
    6: "12:00pm",
    7: "1:15pm",
    8: "2:00pm",
    9: "2:45pm",
    10: "3:45pm",
    11: "4:30pm",
    12: "5:15pm",
    13: "6:30pm",
    14: "7:15pm",
    15: "8:00pm",
    16: "8:45pm"
}



# Dictionary of preference strings and corresponding values
preferencedict = {
    "Available, Prefer": 1,
    "Available, Neutral Preference": 2,
    "Available, Prefer Not": 3,
    "Unavailable": 4
}

# Dictionary of speaker vetos and corresponding values

vetodict = {
    "I would not like to veto any position": 0,
    "1st Speaker": 1,
    "2nd Speaker": 2,
    "3rd Speaker": 3,
}

vetodictinv = {
    0: "has no speaker position vetoes",
    1: "has vetoed 1st speaker",
    2: "has vetoed 2nd speaker",
    3: "has vetoed 3rd speaker"
}

# Person class
# - No preferences or vetoes for Easters trials

class person:
    name = ""
    
    # Speaker veto - 1 for 1st, 2 for 2nd, 3 for 3rd, 0 for no veto
    speakerveto = 0
    
    # Key-value pair for availability - Key is the slot
    # 1 = Prefer
    # 2 = Available
    # 3 = Prefer Not
    # 4 = Unavailable
    availabilities = {}
    
    def __init__(self,name,availabilities,speakerveto,isAllocated):
        self.name = name
        self.availabilities = availabilities
        self.speakerveto = speakerveto
        isAllocated = False
        
        
def importData():
    print("Please note the final allocation may take up to 10 minutes to compute.")
    global speakers
    speakers = []
    global times
    times = {}
    # CSV reading and parsing
    global csv
    csv = pd.read_csv("data.csv")
    csv.head()
    for index,row in csv.iterrows():
        if row[7] != "Adjudicating only": # MARK: Trial type
            avail = {}
            for i in range(14,29): # MARK: Range of trial slots
                avail[i-13] = preferencedict[row[i]] # MARK: Subtraction
            speakers.append(person(row[3],avail,vetodict[row[13]],False)) # MARK: Details

    # Calculate how many timeslots are required
    reqd = int(np.floor(len(speakers)/6))
    for key,value in alltimeslots.iteritems():
        if key <= reqd:
            times[key] = value
    print("A minimum of {} debates is required to distribute {} speakers.".format(reqd,len(speakers)))
    
    '''If the allocator produces an infeasible result there's a good chance it needs more timeslots.
    Manually override times with a higher number of slots if required'''
    
    for t in range(1,reqd):
        times[t] = alltimeslots[t]

    # Form the feasible team set
    global ftset
    global ftsample
    ftset = {}
    ftsample = {}
    totalteams = len(speakers)**3
    index = 1
    print("Forming Feasible Team Set...")
    for speaker1 in speakers:
        for speaker2 in speakers:
            for speaker3 in speakers:
                if speaker1 != speaker2 and speaker1 != speaker3 and speaker2 != speaker3:
                    if speaker1.speakerveto != 1 and speaker2.speakerveto != 2 and speaker3.speakerveto != 3:
                        print("Constructing team {} of {}...".format(index,totalteams))
                        ftset[index] = [speaker1,speaker2,speaker3]
                        index += 1
    
    randomkeys = random.sample(ftset.keys(), len(ftset)/1) # Vary so that a smaller subset is used for the fts
    for k in randomkeys:
        ftsample[k] = ftset[k]
    
    # Costs
    global costs
    costs = {}
    totalcosts = len(ftsample)*len(times)
    print("Calculating costs...")
    index2 = 1
    for key,team in ftsample.iteritems():
        for slot in times.keys():
            costs[key,slot] = 0
            index2 += 1
            print("Calculating cost {} of {}...".format(index2,totalcosts))
    
        # Time preferences
        '''Triallists are made aware that they may be called during "unavailable",
        so an 'unavailable' does not make the cost infinitely high, it just incurs
        a high cost.
        
        As slots become less desirable, the cost increases exponentially rather than
        linearly'''
    
        for speaker in team:
            for slot in times.keys():
                if speaker.availabilities[slot] == 4:
                    cost = grb.GRB.INFINITY
                else:
                    mag = speaker.availabilities[slot]*100
                    exp = mag**2
                    costs[key,slot] += exp
    
def allocate():
    # Optimisation
    a = grb.Model("2019 Easters Trial Allocator")
    x = {}
    for key,value in costs.iteritems():
        x[key] = a.addVar(vtype = grb.GRB.BINARY, obj = value, name = "x_{}".format(key))
        
    # Exactly 2 teams to each timeslot
    for slot in times.keys():
        a.addConstr(sum(x[team,slot] for team in ftsample.keys()) == 2)
        
    # Each team to no more than 1 timeslot
    for key,team in ftsample.iteritems():
        a.addConstr(sum(x[key,slot] for slot in times.keys()) <= 1)

    thirdvetoes = []
    for s in speakers:
        affectedTeams = []
        for key,team in ftsample.iteritems():
            if s in team:
                affectedTeams.append(key)
                
        # Each speaker to no more than 1 debate in 1 timeslot
        '''Can be 0 debates (ie. unallocated) and then appended
        as alternate 3rds'''
        a.addConstr(sum(x[team,slot] for team in affectedTeams for slot in times.keys()) <= 1)
        
        if s.speakerveto == 3:
            thirdvetoes.append(s)
            
    # Speakers who veto 3rd must be in exactly 1 timeslot
    '''3rd vetoes cannot be unallocated and then tacked on as
    alternate 3rds, so they must be allocated in a debate.'''
    for speaker in thirdvetoes:
        affectedTeams2 = []
        for key,team in ftsample.iteritems():
            if speaker in team:
                affectedTeams2.append(key)
        a.addConstr(sum(x[team,slot] for team in affectedTeams2 for slot in times.keys()) == 1)
    
    a.modelSense = grb.GRB.MINIMIZE
    a.optimize()
    
    unallocated = []
    unallocatednames = []
    print(div)
    print("USU EASTERS TRIALS 2019 - DEBATE ALLOCATION")
    print("Speakers in each team are listed in speaking order.")
    print("Sides for each debate should be determined randomly.")
    print(div)
    for key,value in costs.iteritems():
        if x[key].x != 0:
            spk = []
            for s in ftsample[key[0]]:
                spk.append(s.name)
            print("Team {} {} at {}".format(key[0],spk,times[key[1]]))
            for speaker in ftsample[key[0]]:
                speaker.isAllocated = True
    for speaker in speakers:
        if speaker.isAllocated == False:
            unallocated.append(speaker)
            unallocatednames.append(speaker.name)
    print("Total Cost: {}".format(a.objVal))
    print(div)
    
    j = ", "
    print("Unallocated Speakers: {}".format(j.join(unallocatednames)))
    print(div)
    report(unallocated)

def report(speakers):
    for speaker in speakers:
        if speaker not in speakers:
            print("Speaker {} does not exist".format(speaker.name))
        else:
            vetoString= "{} {}.".format(speaker.name, vetodictinv[speaker.speakerveto])

            print("Report for {}".format(speaker.name))
            print(vetoString)
            print("{}'s availability is as follows:".format(speaker.name))
            for slot,availability in speaker.availabilities.iteritems():
                try:
                    # TODO
                    pass
                    #print("{}: {}".format(times[slot],availability))
                except:
                    pass
        print(div)

In [8]:
importData()

Please note the final allocation may take up to 10 minutes to compute.
A minimum of 3 debates is required to distribute 18 speakers.
Forming Feasible Team Set...
Constructing team 1 of 5832...
Constructing team 2 of 5832...
Constructing team 3 of 5832...
Constructing team 4 of 5832...
Constructing team 5 of 5832...
Constructing team 6 of 5832...
Constructing team 7 of 5832...
Constructing team 8 of 5832...
Constructing team 9 of 5832...
Constructing team 10 of 5832...
Constructing team 11 of 5832...
Constructing team 12 of 5832...
Constructing team 13 of 5832...
Constructing team 14 of 5832...
Constructing team 15 of 5832...
Constructing team 16 of 5832...
Constructing team 17 of 5832...
Constructing team 18 of 5832...
Constructing team 19 of 5832...
Constructing team 20 of 5832...
Constructing team 21 of 5832...
Constructing team 22 of 5832...
Constructing team 23 of 5832...
Constructing team 24 of 5832...
Constructing team 25 of 5832...
Constructing team 26 of 5832...
Constructing te

Constructing team 336 of 5832...
Constructing team 337 of 5832...
Constructing team 338 of 5832...
Constructing team 339 of 5832...
Constructing team 340 of 5832...
Constructing team 341 of 5832...
Constructing team 342 of 5832...
Constructing team 343 of 5832...
Constructing team 344 of 5832...
Constructing team 345 of 5832...
Constructing team 346 of 5832...
Constructing team 347 of 5832...
Constructing team 348 of 5832...
Constructing team 349 of 5832...
Constructing team 350 of 5832...
Constructing team 351 of 5832...
Constructing team 352 of 5832...
Constructing team 353 of 5832...
Constructing team 354 of 5832...
Constructing team 355 of 5832...
Constructing team 356 of 5832...
Constructing team 357 of 5832...
Constructing team 358 of 5832...
Constructing team 359 of 5832...
Constructing team 360 of 5832...
Constructing team 361 of 5832...
Constructing team 362 of 5832...
Constructing team 363 of 5832...
Constructing team 364 of 5832...
Constructing team 365 of 5832...
Constructi

Constructing team 1623 of 5832...
Constructing team 1624 of 5832...
Constructing team 1625 of 5832...
Constructing team 1626 of 5832...
Constructing team 1627 of 5832...
Constructing team 1628 of 5832...
Constructing team 1629 of 5832...
Constructing team 1630 of 5832...
Constructing team 1631 of 5832...
Constructing team 1632 of 5832...
Constructing team 1633 of 5832...
Constructing team 1634 of 5832...
Constructing team 1635 of 5832...
Constructing team 1636 of 5832...
Constructing team 1637 of 5832...
Constructing team 1638 of 5832...
Constructing team 1639 of 5832...
Constructing team 1640 of 5832...
Constructing team 1641 of 5832...
Constructing team 1642 of 5832...
Constructing team 1643 of 5832...
Constructing team 1644 of 5832...
Constructing team 1645 of 5832...
Constructing team 1646 of 5832...
Constructing team 1647 of 5832...
Constructing team 1648 of 5832...
Constructing team 1649 of 5832...
Constructing team 1650 of 5832...
Constructing team 1651 of 5832...
Constructing t

Calculating cost 458 of 6774...
Calculating cost 459 of 6774...
Calculating cost 460 of 6774...
Calculating cost 461 of 6774...
Calculating cost 462 of 6774...
Calculating cost 463 of 6774...
Calculating cost 464 of 6774...
Calculating cost 465 of 6774...
Calculating cost 466 of 6774...
Calculating cost 467 of 6774...
Calculating cost 468 of 6774...
Calculating cost 469 of 6774...
Calculating cost 470 of 6774...
Calculating cost 471 of 6774...
Calculating cost 472 of 6774...
Calculating cost 473 of 6774...
Calculating cost 474 of 6774...
Calculating cost 475 of 6774...
Calculating cost 476 of 6774...
Calculating cost 477 of 6774...
Calculating cost 478 of 6774...
Calculating cost 479 of 6774...
Calculating cost 480 of 6774...
Calculating cost 481 of 6774...
Calculating cost 482 of 6774...
Calculating cost 483 of 6774...
Calculating cost 484 of 6774...
Calculating cost 485 of 6774...
Calculating cost 486 of 6774...
Calculating cost 487 of 6774...
Calculating cost 488 of 6774...
Calculat

Calculating cost 1400 of 6774...
Calculating cost 1401 of 6774...
Calculating cost 1402 of 6774...
Calculating cost 1403 of 6774...
Calculating cost 1404 of 6774...
Calculating cost 1405 of 6774...
Calculating cost 1406 of 6774...
Calculating cost 1407 of 6774...
Calculating cost 1408 of 6774...
Calculating cost 1409 of 6774...
Calculating cost 1410 of 6774...
Calculating cost 1411 of 6774...
Calculating cost 1412 of 6774...
Calculating cost 1413 of 6774...
Calculating cost 1414 of 6774...
Calculating cost 1415 of 6774...
Calculating cost 1416 of 6774...
Calculating cost 1417 of 6774...
Calculating cost 1418 of 6774...
Calculating cost 1419 of 6774...
Calculating cost 1420 of 6774...
Calculating cost 1421 of 6774...
Calculating cost 1422 of 6774...
Calculating cost 1423 of 6774...
Calculating cost 1424 of 6774...
Calculating cost 1425 of 6774...
Calculating cost 1426 of 6774...
Calculating cost 1427 of 6774...
Calculating cost 1428 of 6774...
Calculating cost 1429 of 6774...
Calculatin

Calculating cost 2445 of 6774...
Calculating cost 2446 of 6774...
Calculating cost 2447 of 6774...
Calculating cost 2448 of 6774...
Calculating cost 2449 of 6774...
Calculating cost 2450 of 6774...
Calculating cost 2451 of 6774...
Calculating cost 2452 of 6774...
Calculating cost 2453 of 6774...
Calculating cost 2454 of 6774...
Calculating cost 2455 of 6774...
Calculating cost 2456 of 6774...
Calculating cost 2457 of 6774...
Calculating cost 2458 of 6774...
Calculating cost 2459 of 6774...
Calculating cost 2460 of 6774...
Calculating cost 2461 of 6774...
Calculating cost 2462 of 6774...
Calculating cost 2463 of 6774...
Calculating cost 2464 of 6774...
Calculating cost 2465 of 6774...
Calculating cost 2466 of 6774...
Calculating cost 2467 of 6774...
Calculating cost 2468 of 6774...
Calculating cost 2469 of 6774...
Calculating cost 2470 of 6774...
Calculating cost 2471 of 6774...
Calculating cost 2472 of 6774...
Calculating cost 2473 of 6774...
Calculating cost 2474 of 6774...
Calculatin

Calculating cost 3378 of 6774...
Calculating cost 3379 of 6774...
Calculating cost 3380 of 6774...
Calculating cost 3381 of 6774...
Calculating cost 3382 of 6774...
Calculating cost 3383 of 6774...
Calculating cost 3384 of 6774...
Calculating cost 3385 of 6774...
Calculating cost 3386 of 6774...
Calculating cost 3387 of 6774...
Calculating cost 3388 of 6774...
Calculating cost 3389 of 6774...
Calculating cost 3390 of 6774...
Calculating cost 3391 of 6774...
Calculating cost 3392 of 6774...
Calculating cost 3393 of 6774...
Calculating cost 3394 of 6774...
Calculating cost 3395 of 6774...
Calculating cost 3396 of 6774...
Calculating cost 3397 of 6774...
Calculating cost 3398 of 6774...
Calculating cost 3399 of 6774...
Calculating cost 3400 of 6774...
Calculating cost 3401 of 6774...
Calculating cost 3402 of 6774...
Calculating cost 3403 of 6774...
Calculating cost 3404 of 6774...
Calculating cost 3405 of 6774...
Calculating cost 3406 of 6774...
Calculating cost 3407 of 6774...
Calculatin

Calculating cost 4365 of 6774...
Calculating cost 4366 of 6774...
Calculating cost 4367 of 6774...
Calculating cost 4368 of 6774...
Calculating cost 4369 of 6774...
Calculating cost 4370 of 6774...
Calculating cost 4371 of 6774...
Calculating cost 4372 of 6774...
Calculating cost 4373 of 6774...
Calculating cost 4374 of 6774...
Calculating cost 4375 of 6774...
Calculating cost 4376 of 6774...
Calculating cost 4377 of 6774...
Calculating cost 4378 of 6774...
Calculating cost 4379 of 6774...
Calculating cost 4380 of 6774...
Calculating cost 4381 of 6774...
Calculating cost 4382 of 6774...
Calculating cost 4383 of 6774...
Calculating cost 4384 of 6774...
Calculating cost 4385 of 6774...
Calculating cost 4386 of 6774...
Calculating cost 4387 of 6774...
Calculating cost 4388 of 6774...
Calculating cost 4389 of 6774...
Calculating cost 4390 of 6774...
Calculating cost 4391 of 6774...
Calculating cost 4392 of 6774...
Calculating cost 4393 of 6774...
Calculating cost 4394 of 6774...
Calculatin

Calculating cost 5324 of 6774...
Calculating cost 5325 of 6774...
Calculating cost 5326 of 6774...
Calculating cost 5327 of 6774...
Calculating cost 5328 of 6774...
Calculating cost 5329 of 6774...
Calculating cost 5330 of 6774...
Calculating cost 5331 of 6774...
Calculating cost 5332 of 6774...
Calculating cost 5333 of 6774...
Calculating cost 5334 of 6774...
Calculating cost 5335 of 6774...
Calculating cost 5336 of 6774...
Calculating cost 5337 of 6774...
Calculating cost 5338 of 6774...
Calculating cost 5339 of 6774...
Calculating cost 5340 of 6774...
Calculating cost 5341 of 6774...
Calculating cost 5342 of 6774...
Calculating cost 5343 of 6774...
Calculating cost 5344 of 6774...
Calculating cost 5345 of 6774...
Calculating cost 5346 of 6774...
Calculating cost 5347 of 6774...
Calculating cost 5348 of 6774...
Calculating cost 5349 of 6774...
Calculating cost 5350 of 6774...
Calculating cost 5351 of 6774...
Calculating cost 5352 of 6774...
Calculating cost 5353 of 6774...
Calculatin

Calculating cost 6323 of 6774...
Calculating cost 6324 of 6774...
Calculating cost 6325 of 6774...
Calculating cost 6326 of 6774...
Calculating cost 6327 of 6774...
Calculating cost 6328 of 6774...
Calculating cost 6329 of 6774...
Calculating cost 6330 of 6774...
Calculating cost 6331 of 6774...
Calculating cost 6332 of 6774...
Calculating cost 6333 of 6774...
Calculating cost 6334 of 6774...
Calculating cost 6335 of 6774...
Calculating cost 6336 of 6774...
Calculating cost 6337 of 6774...
Calculating cost 6338 of 6774...
Calculating cost 6339 of 6774...
Calculating cost 6340 of 6774...
Calculating cost 6341 of 6774...
Calculating cost 6342 of 6774...
Calculating cost 6343 of 6774...
Calculating cost 6344 of 6774...
Calculating cost 6345 of 6774...
Calculating cost 6346 of 6774...
Calculating cost 6347 of 6774...
Calculating cost 6348 of 6774...
Calculating cost 6349 of 6774...
Calculating cost 6350 of 6774...
Calculating cost 6351 of 6774...
Calculating cost 6352 of 6774...
Calculatin

In [9]:
allocate()

Optimize a model with 2285 rows, 6774 columns and 39594 nonzeros
Variable types: 0 continuous, 6774 integer (6774 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 3e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 370000.00000
Presolve removed 2264 rows and 4416 columns
Presolve time: 0.07s
Presolved: 21 rows, 2358 columns, 9432 nonzeros
Found heuristic solution: objective 280000.00000
Variable types: 0 continuous, 2358 integer (2358 binary)

Root relaxation: objective 9.000000e+04, 46 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 90000.0000    0   17 280000.000 90000.0000  67.9%     -    0s
H    0     0                    90000.000000 90000.0000  0.00%     -    0s

Explored 1 nodes (113 simplex iterations) in 0.16 seconds
Thread count was 4 